In [90]:
import json
from copy import deepcopy

In [3]:
import getpass
from kgforge.core import KnowledgeGraphForge

In [4]:
from bmo.loading import load_ontologies

In [5]:
from rdflib import XSD, Literal, RDFS, URIRef, Graph

In [6]:
ontology_graphs_dict, all_ontology_graphs = load_ontologies('./ontologies/bbp/*.ttl')
graph = ontology_graphs_dict['./ontologies/bbp/brainregion.ttl']

In [16]:
generator = graph.subject_objects(URIRef("https://neuroshapes.org/hasLayerLocationPhenotype"))

In [17]:
gen_list = list(generator)

In [19]:
len(gen_list)

580

In [39]:
gen_list[0]

(rdflib.term.URIRef('http://api.brain-map.org/api/v2/data/Structure/1005'),
 rdflib.term.URIRef('http://purl.obolibrary.org/obo/UBERON_8440003'))

In [59]:
layers_list = [str(item[1]) for item in gen_list]

In [60]:
len(layers_list)

580

In [61]:
set(layers_list)

{'http://purl.obolibrary.org/obo/UBERON_0002313',
 'http://purl.obolibrary.org/obo/UBERON_0005371',
 'http://purl.obolibrary.org/obo/UBERON_0005372',
 'http://purl.obolibrary.org/obo/UBERON_0005390',
 'http://purl.obolibrary.org/obo/UBERON_0005391',
 'http://purl.obolibrary.org/obo/UBERON_0005392',
 'http://purl.obolibrary.org/obo/UBERON_0005393',
 'http://purl.obolibrary.org/obo/UBERON_0005394',
 'http://purl.obolibrary.org/obo/UBERON_0005395',
 'http://purl.obolibrary.org/obo/UBERON_0007637',
 'http://purl.obolibrary.org/obo/UBERON_0007640',
 'http://purl.obolibrary.org/obo/UBERON_8440003',
 'https://bbp.epfl.ch/ontologies/core/bmo/neocortex_layer_6a'}

In [46]:
regs = [int(str(item[0]).split("/")[-1]) for item in gen_list]

In [47]:
len(regs)

580

In [48]:
len(set(regs))

508

In [49]:
my_dict = {i:regs.count(i) for i in regs}
print(my_dict)

In [58]:
for reg in my_dict:
    if my_dict[reg] > 2:
        print(reg)

In [81]:
layer_parent_map = {
    'http://purl.obolibrary.org/obo/UBERON_0002313': ["http://purl.obolibrary.org/obo/UBERON_0002305"],
    'http://purl.obolibrary.org/obo/UBERON_0005371': ["http://purl.obolibrary.org/obo/UBERON_0002305"],
    'http://purl.obolibrary.org/obo/UBERON_0005372': ["http://purl.obolibrary.org/obo/UBERON_0002305"],
    'http://purl.obolibrary.org/obo/UBERON_0005390': ["http://purl.obolibrary.org/obo/UBERON_0002301"],
    'http://purl.obolibrary.org/obo/UBERON_0005391': ["http://purl.obolibrary.org/obo/UBERON_0002301"],
    'http://purl.obolibrary.org/obo/UBERON_0005392': ["http://purl.obolibrary.org/obo/UBERON_0002301"],
    'http://purl.obolibrary.org/obo/UBERON_0005393': ["http://purl.obolibrary.org/obo/UBERON_0002301"],
    'http://purl.obolibrary.org/obo/UBERON_0005394': ["http://purl.obolibrary.org/obo/UBERON_0002301"],
    'http://purl.obolibrary.org/obo/UBERON_0005395': ["http://purl.obolibrary.org/obo/UBERON_0002301"],
    'http://purl.obolibrary.org/obo/UBERON_0007637': ["http://purl.obolibrary.org/obo/UBERON_0002305"],
    'http://purl.obolibrary.org/obo/UBERON_0007640': ["http://purl.obolibrary.org/obo/UBERON_0002305"],
    'http://purl.obolibrary.org/obo/UBERON_8440003': ["http://purl.obolibrary.org/obo/UBERON_0002301"],
    'https://bbp.epfl.ch/ontologies/core/bmo/neocortex_layer_6a': ["http://purl.obolibrary.org/obo/UBERON_0002301"]
}

In [55]:
from pathlib import Path
from voxcell import RegionMap

In [57]:
mba_ld = str(Path("/home/lcristel/Desktop", "mba_hierarchy_ld_v170.json"))
mba = "/gpfs/bbp.cscs.ch/project/proj84/atlas_pipeline_runs/stable_region_ids/mba_hierarchy.json"
#region_map = RegionMap.load_json(mba_ld)
region_map = RegionMap.load_json(mba)

In [86]:
reg_layer_map = {}
for reg_layer in gen_list:
    reg = int(str(reg_layer[0]).split("/")[-1])
    reg_set = region_map.find(reg, attr="id")
    if not reg_set:
        continue
    acronym = region_map.get(reg, attr="acronym", with_ascendants=False)
    layer = str(reg_layer[1])
    if acronym not in reg_layer_map:
        reg_layer_map[acronym] = {
            "layers": [layer],
            "parent_layers": deepcopy(layer_parent_map[layer])}
    else:
        reg_layer_map[acronym]["layers"].append(layer)
        reg_layer_map[acronym]["parent_layers"].extend(layer_parent_map[layer])

In [87]:
reg_layer_map

{'AUDp6b': {'layers': ['http://purl.obolibrary.org/obo/UBERON_8440003'],
  'parent_layers': ['http://purl.obolibrary.org/obo/UBERON_0002301']},
 'SSp-ul6b': {'layers': ['http://purl.obolibrary.org/obo/UBERON_8440003'],
  'parent_layers': ['http://purl.obolibrary.org/obo/UBERON_0002301']},
 'ECT6b': {'layers': ['http://purl.obolibrary.org/obo/UBERON_8440003'],
  'parent_layers': ['http://purl.obolibrary.org/obo/UBERON_0002301']},
 'SSp-bfd6b': {'layers': ['http://purl.obolibrary.org/obo/UBERON_8440003'],
  'parent_layers': ['http://purl.obolibrary.org/obo/UBERON_0002301']},
 'ILA6b': {'layers': ['http://purl.obolibrary.org/obo/UBERON_8440003'],
  'parent_layers': ['http://purl.obolibrary.org/obo/UBERON_0002301']},
 'MOs6b': {'layers': ['http://purl.obolibrary.org/obo/UBERON_8440003'],
  'parent_layers': ['http://purl.obolibrary.org/obo/UBERON_0002301']},
 'VISl6b': {'layers': ['http://purl.obolibrary.org/obo/UBERON_8440003'],
  'parent_layers': ['http://purl.obolibrary.org/obo/UBERON_00

In [88]:
len(reg_layer_map)

402

In [98]:
with open('regions_layers_map.json', 'w') as fp:
    #json.dump(reg_layer_map, fp)
    fp.write(json.dumps(reg_layer_map, indent=4))

In [99]:
TOKEN = getpass.getpass()

In [100]:
forge_staging = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml", endpoint="https://staging.nise.bbp.epfl.ch/nexus/v1", bucket="bbp/atlas", token=TOKEN)

In [101]:
ex_id = "https://bbp.epfl.ch/neurosciencegraph/data/ab591ed4-3377-43c1-9038-253eeb107dc9"

In [102]:
ex_res = forge_staging.retrieve(ex_id)

In [103]:
delattr(ex_res, "generation")

In [104]:
delattr(ex_res, "id")

In [107]:
from kgforge.core.resource import Resource

In [113]:
ex_res.derivation = Resource(type="Derivation",
                             name="Ontology source",
                             id="https://bbpgitlab.epfl.ch/dke/apps/brain-modeling-ontology/-/blob/083e608e5624bf7d6cd047fd4c66e4f7817ed931/ontologies/bbp/brainregion.ttl")

In [124]:
ex_res.name = "regions-to-layers_map"

In [120]:
ex_res.distribution = forge_staging.attach("regions_layers_map.json", content_type="application/json")

In [119]:
ex_res.description = "Mapping of region to layers, derived from https://bbpgitlab.epfl.ch/dke/apps/brain-modeling-ontology/-/blob/develop/ontologies/bbp/brainregion.ttl"

In [121]:
print(ex_res)

{
    context: https://bbp.neuroshapes.org
    type:
    [
        Dataset
        PipelineConfigResource
    ]
    derivation:
    {
        id: https://bbpgitlab.epfl.ch/dke/apps/brain-modeling-ontology/-/blob/083e608e5624bf7d6cd047fd4c66e4f7817ed931/ontologies/bbp/brainregion.ttl
        type: Derivation
        name: Ontology source
    }
    description: Mapping of region to layers, derived from https://bbpgitlab.epfl.ch/dke/apps/brain-modeling-ontology/-/blob/develop/ontologies/bbp/brainregion.ttl
    distribution: LazyAction(operation=Store.upload, args=['regions_layers_map.json', 'application/json', <kgforge.core.forge.KnowledgeGraphForge object at 0x7f7107328910>])
    name: Regions to layers map
}


In [125]:
forge_staging.update(ex_res)

<action> _update_one
<succeeded> True
